In [1]:
import qiskit, sys
import numpy as np
import qtm.qcompilation, qtm.ansatz, qtm.constant
print(qtm.constant.noise_prob)

0.01


In [2]:
num_qubits = 5
num_layers = 2
thetas = np.ones(num_layers*num_qubits*2)

compiler = qtm.qcompilation.QuantumCompilation(
    u = qtm.ansatz.create_polygongraph_ansatz,
    vdagger = qtm.state.create_ghz_state(num_qubits).inverse(),
    optimizer = 'qng_fubini_study',
    loss_func = 'loss_fubini_study',
    thetas = thetas,
    num_layers = num_layers
)

compiler.fit(num_steps=100, verbose = 1)
print("Done!")

Step:   0%|          | 0/100 [00:00<?, ?it/s]/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/numpy/linalg/linalg.py:2154: RuntimeWarning: divide by zero encountered in det
  r = _umath_linalg.det(a, signature=signature)
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/numpy/linalg/linalg.py:2154: RuntimeWarning: invalid value encountered in det
  r = _umath_linalg.det(a, signature=signature)
Step:   1%|          | 1/100 [00:34<56:24, 34.19s/it]

In [9]:
compiler.save(path = './tomography/noise_qng_polygon', text=f'{qtm.constant.noise_prob}_mitigating', save_all = True)